# Text genration with LTSM

In [1]:
import random
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.utils.vis_utils import plot_model
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
#nltk.download('stopwords')

In [2]:
df = pd.read_csv('gotTwitter100k.csv')
df

C:\Users\52999\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,user_id,status_id,created_at,screen_name,text,source,display_text_width,reply_to_status_id,reply_to_user_id,...,statuses_count,favourites_count,account_created_at,verified,profile_url,profile_expanded_url,account_lang,profile_banner_url,profile_background_url,profile_image_url
0,0,x51554079,x1118417386150014976,2019-04-17 07:34:18,moevazquez,👍 on @YouTube: GAME OF THRONES 8x01 Breakdown!...,IFTTT,99,NaN,NaN,...,206478,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...
1,1,x51554079,x1117994592136249344,2019-04-16 03:34:16,moevazquez,👍 on @YouTube: Ups and Downs From Game Of Thro...,IFTTT,77,NaN,NaN,...,206478,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...
2,2,x51554079,x1117987511391981568,2019-04-16 03:06:08,moevazquez,Liked on YouTube: Ups and Downs From Game Of T...,IFTTT,80,NaN,NaN,...,206478,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...
3,3,x51554079,x1118410676475760640,2019-04-17 07:07:38,moevazquez,Liked on YouTube: GAME OF THRONES 8x01 Breakdo...,IFTTT,102,NaN,NaN,...,206478,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...
4,4,x770111944187580416,x1118417347495251968,2019-04-17 07:34:09,whoisScottyP,@MrLegenDarius unpopular opinion: game of thro...,Twitter for iPhone,42,x1118398110810681345,x37078228,...,1602,25651,2016-08-29 04:12:50,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/77011194...,NaN,http://pbs.twimg.com/profile_images/1108193461...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,x927343023398277120,x1117502882687930373,2019-04-14 19:00:24,TandWToday,I’ve never seen an episode of Game of Thrones ...,Twitter for iPhone,109,NaN,NaN,...,1240,4182,2017-11-06 01:13:00,False,https://t.co/S569xOHids,http://travelandwritetoday.com,en,https://pbs.twimg.com/profile_banners/92734302...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1053239525...
99996,99996,x107763369,x1117502879806500866,2019-04-14 19:00:23,NoelleNevala,Currently listening to Game of Thrones soundtr...,Twitter for iPhone,86,NaN,NaN,...,9243,11627,2010-01-23 17:05:45,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/10776336...,http://abs.twimg.com/images/themes/theme5/bg.gif,http://pbs.twimg.com/profile_images/1118223854...
99997,99997,x3142156757,x1117502876996313088,2019-04-14 19:00:22,bmeredith422,Game of Thrones today lets get it,Twitter for iPhone,33,NaN,NaN,...,1200,1411,2015-04-06 22:40:03,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/31421567...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1053508093...
99998,99998,x338411380,x1117502873443753986,2019-04-14 19:00:21,Jonah_Howard_13,If you aren’t watching Game of Thrones tonight...,Twitter for iPhone,100,NaN,NaN,...,12743,5061,2011-07-19 15:00:37,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/33841138...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1107115854...


In [3]:
class PreProcessor:
    
    
    def __init__(self):
        '''
        The PreProcessor class provides different methods to preprocess a string that the method will receive.
        Attributes:
            pos: stores the position that we assign for the lemmatization. Our base case is the position 'noun', because
            at the moment that we call the lemmatize method it is more common and useful to lemmatize in nouns. This
            position could be modified by the user.
            special_chars: stores the characters that we want to remove to the strings in the remove noise method.
            regex_dict: stores the dictionary of the regular expressions that we want to extract.
        
        Methods:
            stemWords: It receives a string and transforms the words to its root form.
            lemmatizeWords: It receives a string and reduces the words to a word existing in the language.
            removeNoise: It receives a string and remove all the characters that are included in the attribute of special
            characters to obtain a cleaned string.
            wordTokenize: It receives a string and splits into words.
            phraseTokenize: It receives a string and splits into phrases.
            textNormalization: It receives a string and removes the regular expressions that we previously defined in 
            the attribute of regex dict to obtain a string with more coherence.
            extractRegex: It receives a string and extract the regula expressions that we defined in the previous attributes.
            cleaning: This method calls the methods that we consider are necessary for the preprocessing of the tweet 
            database.
        '''
        self.pos = 'n'
        self.special_chars = ",.@?!¬-\''=()"
        self.regex_dict = {'Tags' : r'@[A-Za-z0-9]+', 
                      '# symbol' : r'#', 
                      'RT' : r'RT', 
                      'Links' : r'https?://\S+',
                      'Not letters': r'[^A-Za-z\s]+',
                      'Phone' : r'\+[0-9]{12}'}
    
    def stemWords(self, string):
        '''
        Input: String
        Process: Receives a string and call the PorterStemmer function of the nltk library of python to do the stemming 
        process. We call the word tokenize method to split the string and obtain the root of each word, and then when
        the words are stemmed, we join the string again.
        Return: Stemmed string
        '''
        ps = PorterStemmer()
        stem = list(map(ps.stem, self.wordTokenize(string)))
        stemmed = ' '.join(stem)
        return stemmed
    
    def lemmatizeWords(self, string):
        '''
        Input: String
        Process: Receives a string and call the WordNetLemmatizer function of the nltk library of python to do the 
        lemmatizing process which is going to receive the word and the attribute of position. We call the word tokenize
        method to split the string and obtain the word of the language of each word, and then when the words are 
        lemmatized, we join the string again.
        Return: Lemmatized string
        '''
        wnl = WordNetLemmatizer()
        lemm = [wnl.lemmatize(word, self.pos) for word in self.wordTokenize(string)]
        lematized = ' '.join(lemm)
        return lematized
    
    def removeNoise(self, string):
        '''
        Input: String
        Process: Receives the string and makes all the characters lower, then check the string and if there are 
        characters that are part of the attribute of special characters, it replaces the character with nothing and
        finally join the string again.
        Return: cleaned string
        '''
        clean_string = string.lower()
        for char in self.special_chars:
            clean_string = clean_string.replace(char, "")
        splitted = self.wordTokenize(clean_string)
        cleaned = [w.replace(" ", "") for w in splitted if len(w) > 0]
        clean_string = " ".join(cleaned)
        return clean_string
    
    def wordTokenize(self, string):
        '''
        Input: String
        Process: Receives the string and split all the words adding the words to the list, using the word tokenize 
        function of the nltk.
        Return: list of tokenized words
        '''
        tokenized = word_tokenize(string)
        return tokenized
    
    def phraseTokenize(self, string):
        '''
        Input: String
        Process: Receives the string and split the phrases using the reference '. ' and then adding the phrase to the list
        Return: list of tokenized phrases
        '''
        cleaned = string.split('. ')
        return cleaned
    
    def textNormalization(self, string):
        '''
        Input: String
        Process: Receives and splits the string and remove the regular expressions of the string. To make this, 
        it iterates over the attribute that stores the regular expression dictionary.
        Return: String without the regular expresions
        '''
        for key in self.regex_dict.keys():
            string = re.sub(self.regex_dict[key], '', string)
        normalized =  " ".join(self.wordTokenize(string))
        return normalized
    
    def extractRegex(self, string):
        '''
        Input: String
        Process: Receives the string, creates a dictionary to store the regular expressions that we extract of each
        string and calls the method of text nrmalization to also generate a cleaned string that we also will return to
        the user.
        Return: dictionary with the regular expressions that we extract and a cleaned string without the regular expressions
        '''
        dict_found_strings = dict()
        
        for key in self.regex_dict.keys():
            found_strings = re.findall(self.regex_dict[key], string)
            dict_found_strings[key] = found_strings

        replaced_string = self.textNormalization(string = string)
        return dict_found_strings, replaced_string
    
    def cleaning(self, data):
        '''
        Input: Data
        Process: This method call the methods that we consider necessary for the preprocessing of the tweets database.
        In this case we select the text normalization, the stemmatization words, and the remove noise methods. This 
        method receives in this case a column of the dataframe and returns the column of the dataframe preprocesed.
        Return: Preprocessed Data
        '''
        #text Normalization
        data = data.apply(self.textNormalization)
        
        #stem words
        data = data.apply(self.stemWords)
        
        #Remove Noise
        data = data.apply(self.removeNoise)
        
        return data
        

In [4]:
prep = PreProcessor()

In [5]:
t = df[0:1250]

In [6]:
tweets = t['text']

In [7]:
t['cleaning_tweets'] = prep.cleaning(tweets)
t

<ipython-input-7-e7664d9eb2aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t['cleaning_tweets'] = prep.cleaning(tweets)


,Unnamed: 0,user_id,status_id,created_at,screen_name,text,source,display_text_width,reply_to_status_id,reply_to_user_id,...,favourites_count,account_created_at,verified,profile_url,profile_expanded_url,account_lang,profile_banner_url,profile_background_url,profile_image_url,cleaning_tweets
0,0,x51554079,x1118417386150014976,2019-04-17 07:34:18,moevazquez,👍 on @YouTube: GAME OF THRONES 8x01 Breakdown!...,IFTTT,99,NaN,NaN,...,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...,on game of throne x breakdown night king symbo...
1,1,x51554079,x1117994592136249344,2019-04-16 03:34:16,moevazquez,👍 on @YouTube: Ups and Downs From Game Of Thro...,IFTTT,77,NaN,NaN,...,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...,on up and down from game of throne
2,2,x51554079,x1117987511391981568,2019-04-16 03:06:08,moevazquez,Liked on YouTube: Ups and Downs From Game Of T...,IFTTT,80,NaN,NaN,...,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...,like on youtub up and down from game of throne
3,3,x51554079,x1118410676475760640,2019-04-17 07:07:38,moevazquez,Liked on YouTube: GAME OF THRONES 8x01 Breakdo...,IFTTT,102,NaN,NaN,...,36131,2009-06-27 20:58:25,False,https://t.co/UtWjeVMUCT,https://www.facebook.com/moisesvo,es,https://pbs.twimg.com/profile_banners/51554079...,http://abs.twimg.com/images/themes/theme18/bg.gif,http://pbs.twimg.com/profile_images/9788848932...,like on youtub game of throne x breakdown nigh...
4,4,x770111944187580416,x1118417347495251968,2019-04-17 07:34:09,whoisScottyP,@MrLegenDarius unpopular opinion: game of thro...,Twitter for iPhone,42,x1118398110810681345,x37078228,...,25651,2016-08-29 04:12:50,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/77011194...,NaN,http://pbs.twimg.com/profile_images/1108193461...,unpopular opinion game of throne edit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,1245,x58919230,x1118367622054199296,2019-04-17 04:16:34,SenoritaElsy,Maybe we’ve been binge watching a little too m...,Twitter for iPhone,118,NaN,NaN,...,1044,2009-07-21 20:58:13,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/58919230...,http://abs.twimg.com/images/themes/theme9/bg.gif,http://pbs.twimg.com/profile_images/1093758950...,mayb weve been bing watch a littl too much bec...
1246,1246,x809908063666700288,x1118367603662176256,2019-04-17 04:16:29,Tayo79,Game of Thrones Hotline for Confused Fans http...,Twitter Web Client,78,NaN,NaN,...,14761,2016-12-16 23:48:24,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/80990806...,NaN,http://pbs.twimg.com/profile_images/8117274815...,game of throne hotlin for confus fan via
1247,1247,x114851095,x1118367594174611457,2019-04-17 04:16:27,MPare1966,@francisdarche @MikeNiquette @fred_warren “It ...,Twitter for iPad,212,x1118366391327965184,x71484721,...,870,2010-02-16 20:22:51,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/11485109...,http://abs.twimg.com/images/themes/theme14/bg.gif,http://pbs.twimg.com/profile_images/9150493143...,warren it may never reach game of throne legen...
1248,1248,x850489279235018753,x1118367565783490560,2019-04-17 04:16:20,Dellitweets,@latsondheimer NCIS fell off after Ziva and To...,Twitter for Android,82,x1118363087776690176,x50158116,...,723,2017-04-07 23:23:40,False,NaN,NaN,en,https://pbs.twimg.com/profile_banners/85048927...,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8504948273..

In [8]:
text = t['cleaning_tweets'].to_numpy().tolist()

In [9]:
corpus = 0
for element in text:
    c = len(element)
    corpus = corpus + c
    
print("Corpus length:", corpus)

Corpus length: 108193


In [10]:
characters = []
for element in text:
    let = sorted(list(set(element)))
    characters = characters + let
            
chars = list(set(characters))
print("Total chars:", len(chars))

Total chars: 27


In [11]:
char_indices = {c:i for i, c in enumerate(chars)}
indices_char = {i:c for i, c in enumerate(chars)}

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 100
step = 3
sentences = []
next_chars = []
i = 0
while i < (len(text) - maxlen):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
    i += step
print("Number of sequences:", len(sentences))

Number of sequences: 384


In [12]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')

array_x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
array_y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i in range(len(sentences)):
    for t in range(len(sentences[i])):
        array_x[i, t, char_indices[sentences[i][t]]] = 1
    array_y[i, char_indices[next_chars[i]]] = 1

KeyError: 'on game of throne x breakdown night king symbol explain'

## Build the Model

In [13]:
# build the model
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dropout(0.5),
        layers.Dense(len(chars), activation="softmax"),
    ]
)

In [14]:
def Sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## Train the model

In [15]:
model = load_model(f"tweets.v1.modelgenerative.hdf5")

In [16]:
epochs = 7
batch_size = 1024

optimizer = keras.optimizers.RMSprop(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'], run_eagerly=True)
checkpointer = ModelCheckpoint(filepath=f"tweets.v1.modelgenerative.hdf5", monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(array_x, array_y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[checkpointer])

Epoch 1/7
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.2280
Epoch 1: loss improved from inf to 0.00000, saving model to tweets.v1.modelgenerative.hdf5
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.2280 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/7
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.1954
Epoch 2: loss did not improve from 0.00000
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.1954 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/7
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0782
Epoch 3: loss did not improve from 0.00000
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.0782 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/7
1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.1075
Epoch 4: loss did not improve

In [19]:
def generate(model, sentence, diversity):
    generated = ""
    for i in range(100):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = Sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        generated += next_char

    return generated

In [20]:
text = "what a wonderful serie is house of the dragon"
generate(model, text, 2.7)

'xl yhwbjx rnszkmkuumofpwyyvfbnlxdnzj dmeeyweoltn ivnzpi ajicciiwsxestkhzxavmrnidshre ylvi xtxlshxkjd'

In [21]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 128)               79872     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 27)                3483      
                                                                 
Total params: 83,355
Trainable params: 83,355
Non-trainable params: 0
_________________________________________________________________
